<a href="https://colab.research.google.com/github/Naomie25/DI-Bootcamp/blob/main/Week9_Day2_ExerciceXP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exercise 1: Setup and Environment Configuration

In [ ]:
!pip install langchain

In [ ]:
import langchain
print(langchain.__version__)

0.3.27


In [ ]:
LANGSMITH_API_KEY="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"
import getpass
import os

# Enable LangSmith tracing
os.environ["LANGSMITH_TRACING"] = "true"
# Securely input your LangSmith API key
os.environ["LANGSMITH_API_KEY"] = getpass.getpass(prompt="LangSmith API Key: ")

LangSmith API Key: ··········


In [4]:
import getpass
import os
TAVILY_API_KEY  = "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"
# Securely input your Tavily API key
os.environ["TAVILY_API_KEY"] = getpass.getpass(prompt="Tavily API Key: ")

Tavily API Key: ··········


In [5]:
import os

assert os.getenv("LANGSMITH_TRACING") == "true"
assert os.getenv("LANGSMITH_API_KEY") is not None
assert os.getenv("TAVILY_API_KEY") is not None
print("Environment configured successfully!")

Environment configured successfully!


 Exercise 2: Define Tools

In [6]:
!pip install langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [8]:
search = TavilySearchResults()

/tmp/ipython-input-8-1244456812.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults()


In [9]:
search_results = search.run("latest advancements in AI-driven healthcare triage")
print(search_results)

[{'title': 'Why Artificial Intelligence In Healthcare Is Rewriting Medical ...', 'url': 'https://globalrph.com/2025/02/why-artificial-intelligence-in-healthcare-is-rewriting-medical-diagnosis-in-2025/', 'content': 'Looking ahead, market projections indicate explosive growth, with the AI diagnostics sector expected to reach $10.15 billion by 2033. Emerging technologies promise enhanced capabilities through multimodal analysis, automated reporting, and sophisticated pattern recognition. These advancements signal a fundamental shift toward preventative, data-driven healthcare models that prioritize early intervention and personalized treatment strategies. [...] Remote Diagnostic Capabilities:  \n AI-driven telemedicine has become the life-blood of remote healthcare delivery. This technology lets patients access services from home while healthcare professionals track vital signs and step in quickly when needed. People in remote areas can now use AI-powered platforms to: Consult with physic

In [10]:
tools = [search]

In [11]:
tools

[TavilySearchResults(api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********')))]

Exercise 3: Using Language Models

In [12]:
!pip install -qU "langchain[mistralai]"

In [53]:
import getpass
import os
MISTRAL_API_KEY = "Ip6BeFo1SfdytOJ0Nv04tLQ0WLXhygfT"

os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")


Enter API key for Mistral AI: ··········


In [54]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    provider="mistralai",
    model="mistral-7b-instruct",  # or your chosen Mistral model
    api_key=os.environ["MISTRAL_API_KEY"]
)

/tmp/ipython-input-54-3365882825.py:3: UserWarning: WARNING! provider is not default parameter.
                provider was transferred to model_kwargs.
                Please confirm that provider is what you intended.
  model = init_chat_model(


In [55]:
from langchain_core.messages import HumanMessage

msg = HumanMessage(content="Tell me a fun fact about space exploration.")

In [56]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.messages import HumanMessage
import os

# 🔑 Set your Mistral API key
os.environ["MISTRAL_API_KEY"] = "DRtpspFMaJTRnCX5h16boN6Mi1okoAEa"  # <-- replace with real key

# ✅ Use a valid model name
model = ChatMistralAI(model="mistral-small-latest")

# 🗨️ Create a message
msg = HumanMessage(content="Tell me a fun fact about space exploration.")

# ✅ Use .invoke()
response = model.invoke([msg])

# 🖨️ Print the response
print(response.content)


Here’s a fun fact about space exploration:

**The Voyager 1 spacecraft, launched in 1977, is the farthest human-made object from Earth and is currently in interstellar space—the space between stars!** It carries a golden record with sounds and images of Earth, including greetings in 55 languages, music from different cultures, and even the sound of a kiss. If aliens ever find it, they’ll get a cosmic mixtape from humanity!

Bonus fact: Voyager 1’s power will run out around 2025, but it will keep drifting through the Milky Way for billions of years—long after Earth is gone. 🚀✨


Exercise 4: Tool Binding and Response Inspection

In [57]:
model_with_tools = model.bind_tools(tools)

In [58]:
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool

# 1. Define a dummy tool (for example purposes)
@tool
def get_weather(location: str) -> str:
    """Returns the current weather for a given location."""
    return f"The weather in {location} is sunny and 25°C."

# 2. List of tools
tools = [get_weather]

# 3. Bind tools to the model
model_with_tools = model.bind_tools(tools)

# 4. Use .invoke() instead of calling like a function
response = model_with_tools.invoke([HumanMessage(content="What’s the weather like today in Paris?")])

# 5. Print response and tool call info
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


ContentString: 
ToolCalls: [{'name': 'get_weather', 'args': {'location': 'Paris'}, 'id': 'at7gyRhmg', 'type': 'tool_call'}]


Exercise 5: Create the Agent

In [59]:
!pip install -U langgraph

In [60]:
from langgraph.prebuilt import create_react_agent

In [61]:
agent_executor = create_react_agent(
    model=model,     # ✅ use 'model', not 'llm'
    tools=tools
)


In [62]:
print(agent_executor)

Exercise 7: Streaming Messages

In [66]:
from langchain_core.messages import HumanMessage

In [67]:
messages = [HumanMessage(content="Find the top 3 recent AI papers on medical imaging.")]

In [68]:
stream_iterator = agent_executor.stream(
    {"messages": messages},
    stream_mode="auto"
)

In [69]:
for step in stream_iterator:
    # Each step contains a list of messages; we print the first one
    step["messages"][0].pretty_print()